In [1]:
from lxml import etree
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from matplotlib.patches import Polygon
import os
from bs4 import BeautifulSoup
import re
from collections import Counter
import re
import pandas as pd

In [2]:
from bs4 import BeautifulSoup
import os

path = '/home/noe/Documents/2022-2023/ENC/Mémoire/Guérin/Matenia/Noe/testSortie.xml'

with open(path) as f:
    soup = BeautifulSoup(f,'xml')

textes = [tag.get_text() for tag in soup.find_all('body')]


In [3]:
tree = ET.parse(path)
root = tree.getroot()

liste_textes = []
liste_div = []
for text in root.findall(".//{http://www.tei-c.org/ns/1.0}text"):
    for div in text.findall(".//{http://www.tei-c.org/ns/1.0}div"):
        if 'lat' in div.get('{http://www.w3.org/XML/1998/namespace}lang',''):
            liste_textes.append(text)
            liste_div.append(div)

n = []
ID = []

for text in liste_textes:
    n.append(text.get('n'))
    ID. append(text.get('{http://www.w3.org/XML/1998/namespace}id'))
n_fin = list(filter(lambda x: x is not None, n))
ID_fin = list(filter(lambda x: x is not None, ID))

In [4]:
path2 = '/home/noe/Documents/2022-2023/ENC/Mémoire/Guérin/Matenia/Textes/'
os.chdir(path2)

In [5]:
tomes = []
for i in ID_fin:
    tomes.extend(re.findall('[a-z]{4}[0-9]{1,2}_\d{4}',i))
num = []
for i in tomes:
    num.append(i.split('_')[1])
numéros = [s.lstrip('0') for s in num]

In [6]:
dates = []

for i in numéros:
    with open(f'text_{i}.xml','r') as file:
        data = file.read()
    soup = BeautifulSoup(data,'xml')
    date = soup.find('date')
    dates.append(str(date))

In [7]:
test = []
for i in range(len(dates)):
    if dates[i] == 'None':
        dates[i] = '0000'
    x = re.findall('\d{4}',dates[i])
    test.append(x)

In [8]:
final = []
for i in range(len(test)):
    final.append(test[i][0])

In [9]:
dico = {ID_fin[i]: textes[i] for i in range(len(ID_fin))}

In [10]:
df = pd.DataFrame(dico.items())

In [11]:
colnames = ['id','text']
df.columns = colnames

In [12]:
def clean_text(string):
    first = re.sub("\n *"," ",string)
    second = re.sub(' *',' ',first)
    return first

In [13]:
df['text'] = df['text'].apply(clean_text)

In [14]:
df['dates'] = final

In [15]:
df.head()['text'][0]

'   Philippus, et, universis et cetera Cordi habentes, ut condecet, inimicorum nequicias et impulsus totis conatibus repellendo,  quietem et pacem regni nostri et fidelium incolarum ejusdem, auxiliante Deo precipue et ipsorum mediante subditorum subsidio, procurare, ac providere ne mercatores et alii subditi nostri in suis mercaturis et negociis aliis impediri valeant vel exponere se periculis quibuscunque ; notum facimus quod nos, licet ad hoc personaliter intendere proponamus cucum viis et modis pluribus exquisitis, finaliter deliberato cumcum peritis in talibus etet quamplurimum famosis consilio, certam procedendi viam supper hoc providimus utilem et salubrem, quam dilectis et fidelibus nostris S. de Ruppe Choardi et J. Vigeri militibus quos ad hoc ad Pictavensem mittimus senescalliam, apperuimus, per eos vel eorum alterum cum senescallo ejusdem senescallie, ubi et quibus expedire viderint, declarandam, videlicet de sub veniendo vobis in hoc facto, quod verisimiliter omnes tangit, d

In [16]:
def clean_text(string):
    first = re.sub("\n","",string)
    new_string = re.sub("[0-9]. ",'',first)
    new_string_2 = re.sub('- ','',new_string)
    final = re.sub(' . ',' ', new_string_2)
    final2 = re.sub('-, ','',final)
    final3 = re.sub('[0-9]','',final2)
    return final3

In [17]:
file = "/home/noe/Documents/2022-2023/ENC/Mémoire/Livres_OCR_par_chartes/PDV_lemmes.xlsx"

In [18]:
pdv = pd.read_excel(file)

In [19]:
df_50 = pd.DataFrame(pdv['textes'][50].split('.'))

In [20]:
lol = df['text'][0].strip()

In [21]:
text = re.sub(r"\s+", " ", lol)

In [22]:
def clean_column(text):
    text = text.strip()
    text = re.sub(r"\s+", " ", text)
    return text

In [23]:
df['text'] = df['text'].apply(clean_column)

In [34]:
path = '/home/noe/Documents/2022-2023/ENC/Mémoire/Guérin/Matenia/textes/'
os.chdir(path)   

In [37]:
from typing import List
from pie_extended.cli.utils import get_tagger, get_model, download

# In case you need to download
do_download = False
if do_download:
    for dl in download("lasla"):
        x = 1

# model_path allows you to override the model loaded by another .tar
model_name = "lasla"
tagger = get_tagger(model_name, batch_size=256, device="cpu", model_path=None)



i = 217
while i <= 358:
    
    liste_lemmes = []
    
    sentences: List[str] = [df['text'][i]]
    # Get the main object from the model (: data iterator + postprocesor
    from pie_extended.models.lasla.imports import get_iterator_and_processor
    for sentence_group in sentences:
        iterator, processor = get_iterator_and_processor()
        lemma = tagger.tag_str(sentence_group, iterator=iterator, processor=processor) 

    for elm in lemma:
        if elm['pos'] == 'ADJqua' or elm['pos'] == 'VER' or elm['pos'] == 'NOMcom' or elm['pos'] == 'NOMpro' or elm['pos'] == 'ADJcar' or elm['pos'] == 'ADJord' or elm['pos'] == 'ADJmul' or elm['pos'] == 'ADJdis':
            liste_lemmes.append(elm['lemma'])
    texte = ' '.join(liste_lemmes)
    
    with open(f'texte_{i+1}.txt','w') as file:
        file.write(texte)
    i+=1


KeyError: 358

In [48]:
from typing import List
from pie_extended.cli.utils import get_tagger, get_model, download

# In case you need to download
do_download = False
if do_download:
    for dl in download("lasla"):
        x = 1

# model_path allows you to override the model loaded by another .tar
model_name = "lasla"
tagger = get_tagger(model_name, batch_size=256, device="cpu", model_path=None)


i = 214
liste_lemmes = []

sentences: List[str] = [df['text'][i]]
# Get the main object from the model (: data iterator + postprocesor
from pie_extended.models.lasla.imports import get_iterator_and_processor
for sentence_group in sentences:
    iterator, processor = get_iterator_and_processor()
    lemma = tagger.tag_str(sentence_group, iterator=iterator, processor=processor) 

for elm in lemma:
    if elm['pos'] == 'ADJqua' or elm['pos'] == 'VER' or elm['pos'] == 'NOMcom' or elm['pos'] == 'NOMpro' or elm['pos'] == 'ADJcar' or elm['pos'] == 'ADJord' or elm['pos'] == 'ADJmul' or elm['pos'] == 'ADJdis':
        liste_lemmes.append(elm['lemma'])
texte = ' '.join(liste_lemmes)
with open(f'texte_{i+1}.txt','w') as file:
    file.write(texte)

In [52]:
with open('a_changer','r') as file:
    data = file.readlines()

In [53]:
texte = ' '.join(data)

In [56]:
i = 0
lemmas = []

while i < 358:
    with open(f'texte_{i+1}.txt','r') as file:
        data = file.readlines()
    lemmas.extend(data)
    i+=1

In [61]:
df['lemma'] = lemmas

In [63]:
df.to_excel('Guérin_dataframe.xlsx')